# Homework 12

https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning

* Implement a mini batch functionality to train a regressor.
    - (Optional) If anyone want to do this in a pipeline can do this: https://koaning.github.io/tokenwiser/api/pipeline.html

* Save model, load the model again and test it on `X_test` __Do NOT commit the pickle file__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    df = df.sample(5000, random_state=100).reset_index(drop=True)
    
    y = df['sellingprice']
    df.drop('sellingprice', axis=1, inplace=True)
    X = df
    
    return X,y

def partial_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
   
    while(True):
        yield df.sample(100).reset_index(drop=True)
        
gen = partial_df()

In [4]:
X_test, y_test = test_df()

In [5]:
# each time you call this you will get a new slice of the dataframe.
next(gen)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2012,Nissan,Altima,2.5 S,Coupe,automatic,1n4al2ep1cc193202,fl,2.8,41023.0,white,beige,nissan-infiniti lt,13050,13400,Tue Mar 03 2015 01:30:00 GMT-0800 (PST)
1,2013,Ford,Focus,SE,Sedan,automatic,1fadp3f22dl105351,oh,4.6,13377.0,black,—,"ford motor credit company,llc",12350,12500,Tue Jan 06 2015 09:00:00 GMT-0800 (PST)
2,2012,Toyota,RAV4,Base,SUV,automatic,2t3bf4dv0cw257387,ma,2.8,7103.0,red,tan,capital one auto finance,17900,16600,Tue Jan 20 2015 01:30:00 GMT-0800 (PST)
3,2007,Dodge,Magnum,SE,Wagon,automatic,2d4fv47v17h749841,fl,2.2,114032.0,silver,gray,repo remarketing/florida hospital credit union,5025,5300,Wed Feb 25 2015 01:05:00 GMT-0800 (PST)
4,2014,Ford,Fusion,SE,sedan,automatic,3fa6p0h7xer396779,tx,1.9,13339.0,silver,black,"ford motor credit company,llc",14350,11000,Wed Jun 10 2015 03:30:00 GMT-0700 (PDT)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2014,Mercedes-Benz,E-Class,E350 Sport 4MATIC,Sedan,automatic,wddhf8jb7ea866866,oh,4.7,11579.0,black,black,mercedes-benz financial services,41200,41000,Tue Feb 24 2015 01:30:00 GMT-0800 (PST)
96,2007,Jeep,Wrangler,Unlimited Sahara,SUV,manual,1j4ga59167l119551,mi,3.5,85890.0,black,black,stratford hyundai,16250,16500,Thu Feb 19 2015 01:30:00 GMT-0800 (PST)
97,2011,Chevrolet,Equinox,LS,SUV,automatic,2cnalbec4b6250048,wi,4.3,87687.0,gray,gray,sutton ford/ge,9825,9700,Wed Jan 21 2015 02:00:00 GMT-0800 (PST)
98,2013,Hyundai,Sonata,GLS,Sedan,automatic,5npeb4ac9dh809313,nj,3.9,31303.0,black,black,merchants automotive group,12750,12500,Wed Jan 28 2015 02:00:00 GMT-0800 (PST)


In [6]:
numerical_features = ['year', 'odometer']
categorical_features = ['make', 'model', 'trim', 'body', 'transmission', 'vin', 'state', 'condition', 'color', 'interior', 'seller', 'mmr', 'saledate']

In [7]:
numerical_preprocessor = StandardScaler()
categorical_preprocessor = OneHotEncoder(handle_unknown='ignore')

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_preprocessor, numerical_features),
        ('categorical', categorical_preprocessor, categorical_features)
    ])

In [8]:
regressor = make_pipeline(preprocessor, SGDRegressor(learning_rate='constant', eta0=0.01))

batch_size = 100
num_epochs = 10
for epoch in range(num_epochs):
    for i in range(len(X_test) // batch_size):
        df_batch = next(gen)
        X_batch = df_batch.drop('sellingprice', axis=1)
        y_batch = df_batch['sellingprice']

        regressor.fit(X_batch, y_batch)

..........